In [200]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.dummy import DummyClassifier
from sklearn.metrics import mean_squared_error
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix,precision_score, recall_score
from sklearn.metrics import classification_report
# for ensemble method use
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_selection import SelectKBest,f_classif
from sklearn.feature_selection import chi2
from imblearn.over_sampling import SMOTE
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer


#TO DO-
#One-R
# sampling SOMTE


# height removed
Test_Target = ["SUSPECT_ARRESTED_FLAG", "STOP_WAS_INITIATED",
                 "OBSERVED_DURATION_MINUTES", "SUSPECTED_CRIME_DESCRIPTION",
                 "STOP_DURATION_MINUTES", "FIREARM_FLAG",
                 "SUSPECT_RACE_DESCRIPTION",
                 "DEMEANOR_OF_PERSON_STOPPED", "SUSPECT_BODY_BUILD_TYPE","FRISKED_FLAG","SEARCHED_FLAG","STOP_LOCATION_BORO_NAME"]
Test1 = pd.read_excel("sqf2022.xlsx", usecols=Test_Target)



In [201]:
Feature_test=["FIREARM_FLAG","FRISKED_FLAG","SEARCHED_FLAG"]

Test1['FIREARM_FLAG'] = Test1['FIREARM_FLAG'].replace('(null)', 'No')
Test1.replace('(null)', np.nan, inplace=True)

lb = LabelBinarizer()
Test1["SUSPECT_ARRESTED_FLAG"] = lb.fit_transform(Test1["SUSPECT_ARRESTED_FLAG"])
Test1["FIREARM_FLAG"] = lb.fit_transform(Test1["FIREARM_FLAG"])
Test1["FRISKED_FLAG"] = lb.fit_transform(Test1["FRISKED_FLAG"])
Test1["SEARCHED_FLAG"] = lb.fit_transform(Test1["SEARCHED_FLAG"])


Test1["SUSPECT_ARRESTED_FLAG"].fillna(Test1["SUSPECT_ARRESTED_FLAG"].median,inplace=True)
Test1["FIREARM_FLAG"].fillna(Test1["FIREARM_FLAG"].median,inplace=True)
Test1["FRISKED_FLAG"].fillna(Test1["FRISKED_FLAG"].median,inplace=True)
Test1["SEARCHED_FLAG"].fillna(Test1["SEARCHED_FLAG"].median,inplace=True)

In [202]:
#One Hot-Encoding
OneHot_Encode_feature = ["STOP_WAS_INITIATED","SUSPECTED_CRIME_DESCRIPTION"]
Label_En = OneHotEncoder(handle_unknown="ignore")
Feature_Encoded = Label_En.fit_transform(Test1[OneHot_Encode_feature])
Label_df = pd.DataFrame(Feature_Encoded.toarray(), columns=Label_En.get_feature_names_out(OneHot_Encode_feature))


In [203]:
#label encoding
LE=LabelEncoder()
Test1['SUSPECT_BODY_BUILD_TYPE']=LE.fit_transform(Test1["SUSPECT_BODY_BUILD_TYPE"])
Test1['SUSPECT_BODY_BUILD_TYPE'].fillna(Test1['SUSPECT_BODY_BUILD_TYPE'].mean, inplace=True)

In [204]:
# Find a way to limit, max featrue, 



Test1['DEMEANOR_OF_PERSON_STOPPED'].fillna('neutral', inplace=True)
texts=Test1['DEMEANOR_OF_PERSON_STOPPED'].tolist()
stop_words = ['the', 'is', 'and', 'to', 'of']

# # Initialize the CountVectorizer
vectorizer = CountVectorizer(max_features=10,stop_words=stop_words)

# # Fit and transform the texts using the CountVectorizer
X = vectorizer.fit_transform(texts)

# # Get the feature names
feature_names = vectorizer.get_feature_names()

# # Convert the vectorized data to a DataFrame
vectorized_data = pd.DataFrame(X.toarray(), columns=feature_names)

# # Print the vectorized data
print(vectorized_data)


       agitated  angry  annoyed  calm  compliant  cooperative  nervous  \
0             0      0        0     0          0            0        0   
1             0      0        0     0          0            0        0   
2             0      0        0     0          0            0        1   
3             0      0        0     1          0            0        0   
4             0      0        0     1          0            0        0   
...         ...    ...      ...   ...        ...          ...      ...   
15097         0      0        0     0          0            0        0   
15098         0      0        0     1          0            0        0   
15099         0      0        0     0          1            0        0   
15100         0      0        1     0          0            0        0   
15101         0      0        1     0          1            0        0   

       neutral  normal  upset  
0            0       0      0  
1            1       0      0  
2            0 

/Users/shion/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [205]:
nlpp = pd.read_excel("sqf2022.xlsx", usecols=["DEMEANOR_OF_PERSON_STOPPED"])
print(len(nlpp))
nlp = nlpp.fillna(value='neutral')
print(len(nlp))

# Assuming df is your DataFrame and 'column_name' is the name of the column you want to check
null_count = nlp['DEMEANOR_OF_PERSON_STOPPED'].isnull().sum()
print(len(nlp))
print(f"Number of null values in 'DEMEANOR_OF_PERSON_STOPPED': {null_count}")
data = nlp['DEMEANOR_OF_PERSON_STOPPED'].tolist()
analyzer = SentimentIntensityAnalyzer()
new_data = [analyzer.polarity_scores(element)['compound'] for element in data]
new = [(pair, compound) for pair, compound in zip(data, new_data)]
# Create DataFrame from list of tuples
final_df = pd.DataFrame(new, columns=['DEMEANOR_OF_PERSON_STOPPED', 'COMPOUND_SENTIMENT'])


15102
15102
15102
Number of null values in 'DEMEANOR_OF_PERSON_STOPPED': 0


In [206]:

#Test1 = pd.concat([Test1, Label_df,vectorized_data], axis=1)
Test1 = pd.concat([Test1, Label_df], axis=1)
#Test1=pd.concat([Test1,vectorized_data],axis=1)
#Test1 = pd.concat([Test1,Label_df, final_df], axis=1)


#create instance of minmax scale
scaler = MinMaxScaler(feature_range=(0,1))

# fir the data in 
#Test1["COMPOUND_SENTIMENT"]= scaler.fit_transform(Test1['COMPOUND_SENTIMENT'])
#cat_feature=list(Label_df.columns)+Feature_test+list(vectorized_data.columns)

#cat_feature=list(Label_df.columns)+Feature_test+["COMPOUND_SENTIMENT"]

cat_feature=list(Label_df.columns)+Feature_test

# drop the original categorical feature column
#Test1 = Test1.drop(columns=OneHot_Encode_feature)
#Test1= Test1.drop(columns="DEMEANOR_OF_PERSON_STOPPED")

Test1.to_csv('output.csv', index=False)





train_data,test_data=train_test_split(Test1,test_size=0.2,random_state=42)


#classi
X_train = train_data[cat_feature] # training features_for Random forest
y_train = train_data['SUSPECT_ARRESTED_FLAG'] # training target variable for random forest

X_test = test_data[cat_feature] # testing features
y_test = test_data['SUSPECT_ARRESTED_FLAG'] # testing target variable


smote = SMOTE()
X_train_oversampled, y_train_oversampled = smote.fit_resample(X_train, y_train)

In [207]:
Model_N=MultinomialNB()
Model_N.fit(X_train,y_train)
N_prediction=Model_N.predict(X_test)
train_N_prediction=Model_N.predict(X_train)
N_train_accuracy=accuracy_score(y_train,train_N_prediction)
N_accuracy=accuracy_score(y_test,N_prediction)

print('Accuracy:', N_accuracy)
print('Train_Accuracy:', N_train_accuracy)
print('Confusion Matrix:')
print(confusion_matrix(y_test, N_prediction))

precision = precision_score(y_test, N_prediction, average='macro')
recall = recall_score(y_test, N_prediction, average='macro')

print('Precision:', precision)
print('Recall:', recall)

classification_report = classification_report(y_test, N_prediction)
print('Classification Report:')
print(classification_report)

#add percision and recall, get the marco avg and weighted avg

Accuracy: 0.8391261171797418
Train_Accuracy: 0.8370168032447645
Confusion Matrix:
[[1858  186]
 [ 300  677]]
Precision: 0.8227275802559556
Recall: 0.8009697604592516
Classification Report:
              precision    recall  f1-score   support

           0       0.86      0.91      0.88      2044
           1       0.78      0.69      0.74       977

    accuracy                           0.84      3021
   macro avg       0.82      0.80      0.81      3021
weighted avg       0.84      0.84      0.84      3021



In [208]:
from sklearn.metrics import classification_report
Model_N_Over = MultinomialNB()
Model_N_Over.fit(X_train_oversampled, y_train_oversampled)
N_prediction_Over = Model_N_Over.predict(X_test)
train_N_prediction_Over = Model_N_Over.predict(X_train_oversampled)
N_train_accuracy_Over = accuracy_score(y_train_oversampled, train_N_prediction_Over)
N_accuracy_Over = accuracy_score(y_test, N_prediction_Over)

print('Train_Accuracy:', N_train_accuracy_Over)
print('Accuracy:', N_accuracy_Over)
print(confusion_matrix(y_test, N_prediction_Over))
precision_Over = precision_score(y_test, N_prediction_Over, average='macro')
recall_Over = recall_score(y_test, N_prediction_Over, average='macro')

print('Precision:', precision_Over)
print('Recall:', recall_Over)

classification_report = classification_report(y_test, N_prediction_Over)
print('Classification Report:')
print(classification_report)

Train_Accuracy: 0.791898922333705
Accuracy: 0.7931148626282688
[[1555  489]
 [ 136  841]]
Precision: 0.7759525217538228
Recall: 0.8107807858635104
Classification Report:
              precision    recall  f1-score   support

           0       0.92      0.76      0.83      2044
           1       0.63      0.86      0.73       977

    accuracy                           0.79      3021
   macro avg       0.78      0.81      0.78      3021
weighted avg       0.83      0.79      0.80      3021



In [209]:
from sklearn.metrics import classification_report

model_R = RandomForestClassifier(random_state=42)


model_R.fit(X_train,y_train)
Prediction=model_R.predict(X_test)

train_Prediction =model_R.predict(X_train)


train_accuracy=accuracy_score(y_train,train_Prediction)

accuracy = accuracy_score(y_test, Prediction)


print('Train_Accuracy:', train_accuracy)
print('Accuracy:', accuracy)


print(confusion_matrix(y_test, Prediction))
precision = precision_score(y_test, Prediction, average='macro')
recall = recall_score(y_test, Prediction, average='macro')

print('Precision:', precision)
print('Recall:', recall)

classification_report = classification_report(y_test, Prediction)
print('Classification Report:')
print(classification_report)


Train_Accuracy: 0.8673950831884778
Accuracy: 0.8679245283018868
[[1877  167]
 [ 232  745]]
Precision: 0.8534406116642959
Recall: 0.8404179193865962
Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.92      0.90      2044
           1       0.82      0.76      0.79       977

    accuracy                           0.87      3021
   macro avg       0.85      0.84      0.85      3021
weighted avg       0.87      0.87      0.87      3021



In [210]:
from sklearn.metrics import classification_report
model_R_Over = RandomForestClassifier(random_state=42)
model_R_Over.fit(X_train_oversampled,y_train_oversampled)
Prediction_Over=model_R_Over.predict(X_test)

train_Prediction_Over =model_R_Over.predict(X_train_oversampled)


train_accuracy_Over=accuracy_score(y_train_oversampled,train_Prediction_Over)

accuracy_Over = accuracy_score(y_test, Prediction_Over)


print('Train_Accuracy:', train_accuracy_Over)
print('Accuracy:', accuracy_Over)


print(confusion_matrix(y_test, Prediction_Over))
precision_Over = precision_score(y_test, Prediction_Over, average='macro')
recall_Over = recall_score(y_test, Prediction_Over, average='macro')

print('Precision:', precision_Over)
print('Recall:', recall_Over)

classification_report = classification_report(y_test, Prediction_Over)
print('Classification Report:')
print(classification_report)


Train_Accuracy: 0.8409513192121888
Accuracy: 0.8579940417080437
[[1812  232]
 [ 197  780]]
Precision: 0.8363461262264473
Recall: 0.8424296991268851
Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.89      0.89      2044
           1       0.77      0.80      0.78       977

    accuracy                           0.86      3021
   macro avg       0.84      0.84      0.84      3021
weighted avg       0.86      0.86      0.86      3021



In [211]:
from sklearn.metrics import classification_report
logreg = LogisticRegression(max_iter=1000,random_state=42)
logreg.fit(X_train, y_train)

log_predict=logreg.predict(X_test)
log_score=accuracy_score(y_test,log_predict)

train_log_predict=logreg.predict(X_train)
train_log_score=accuracy_score(y_train,train_log_predict)

print("Accuracy:",log_score)
print("Train Accuract:",train_log_score)

print(confusion_matrix(y_test, log_predict))
precision = precision_score(y_test, log_predict, average='macro')
recall = recall_score(y_test, log_predict, average='macro')

print('Precision:', precision)
print('Recall:', recall)

classification_report = classification_report(y_test, log_predict)
print('Classification Report:')
print(classification_report)

Accuracy: 0.8659384309831182
Train Accuract: 0.8611042132273818
[[1877  167]
 [ 238  739]]
Precision: 0.8515718691778998
Recall: 0.8373472950263097
Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.92      0.90      2044
           1       0.82      0.76      0.78       977

    accuracy                           0.87      3021
   macro avg       0.85      0.84      0.84      3021
weighted avg       0.86      0.87      0.86      3021



In [212]:

from sklearn.metrics import classification_report

logreg_Over = LogisticRegression(max_iter=1000, random_state=42)
logreg_Over.fit(X_train_oversampled, y_train_oversampled)
log_predict_Over = logreg_Over.predict(X_test)
log_score_Over = accuracy_score(y_test, log_predict_Over)

train_log_predict_Over = logreg_Over.predict(X_train_oversampled)
train_log_score_Over = accuracy_score(y_train_oversampled, train_log_predict_Over)

print("Accuracy:", log_score_Over)
print("Train Accuracy:", train_log_score_Over)

print(confusion_matrix(y_test, log_predict_Over))
precision_Over = precision_score(y_test, log_predict_Over, average='macro')
recall_Over = recall_score(y_test, log_predict_Over, average='macro')

print('Precision:', precision_Over)
print('Recall:', recall_Over)

classification_report = classification_report(y_test, log_predict_Over)
print('Classification Report:')
print(classification_report)


Accuracy: 0.864614366103939
Train Accuracy: 0.8371113588504893
[[1847  197]
 [ 212  765]]
Precision: 0.8461278460064279
Recall: 0.843314782061785
Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.90      0.90      2044
           1       0.80      0.78      0.79       977

    accuracy                           0.86      3021
   macro avg       0.85      0.84      0.84      3021
weighted avg       0.86      0.86      0.86      3021



In [213]:
from sklearn.metrics import classification_report

Dec_model=DecisionTreeClassifier(random_state=42)
Dec_model.fit(X_train,y_train)
Dec_predict=Dec_model.predict(X_test)
Train_Dec_predict=Dec_model.predict(X_train)
Dec_accuracy=accuracy_score(y_test,Dec_predict)
Train_Dec_accuracy=accuracy_score(y_train,Train_Dec_predict)
print("Decision Tree score: ",Dec_accuracy)
print("Train_Decision Tree score: ",Train_Dec_accuracy)

print(confusion_matrix(y_test, Dec_predict))
precision = precision_score(y_test, Dec_predict, average='macro')
recall = recall_score(y_test, Dec_predict, average='macro')

print('Precision:', precision)
print('Recall:', recall)

classification_report = classification_report(y_test, Dec_predict)
print('Classification Report:')
print(classification_report)




Decision Tree score:  0.8652763985435287
Train_Decision Tree score:  0.8673950831884778
[[1880  164]
 [ 243  734]]
Precision: 0.8514556343871922
Recall: 0.8355222965786474
Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.92      0.90      2044
           1       0.82      0.75      0.78       977

    accuracy                           0.87      3021
   macro avg       0.85      0.84      0.84      3021
weighted avg       0.86      0.87      0.86      3021



In [214]:
from sklearn.metrics import classification_report

Dec_model_Over = DecisionTreeClassifier(random_state=42)
Dec_model_Over.fit(X_train_oversampled, y_train_oversampled)

# Make predictions on the testing data
Dec_predict_Over = Dec_model_Over.predict(X_test)

# Make predictions on the training data (oversampled)
Train_Dec_predict_Over = Dec_model_Over.predict(X_train_oversampled)

# Calculate accuracy scores
Dec_accuracy_Over = accuracy_score(y_test, Dec_predict_Over)
Train_Dec_accuracy_Over = accuracy_score(y_train_oversampled, Train_Dec_predict_Over)

print("Decision Tree score on testing data: ", Dec_accuracy_Over)
print("Decision Tree score on training data (oversampled): ", Train_Dec_accuracy_Over)

print(confusion_matrix(y_test, Dec_predict_Over))
precision_Over = precision_score(y_test, Dec_predict_Over, average='macro')
recall_Over = recall_score(y_test, Dec_predict_Over, average='macro')

print('Precision:', precision_Over)
print('Recall:', recall_Over)

classification_report = classification_report(y_test, Dec_predict_Over)
print('Classification Report:')
print(classification_report)


Decision Tree score on testing data:  0.8560079443892751
Decision Tree score on training data (oversampled):  0.8409513192121888
[[1812  232]
 [ 203  774]]
Precision: 0.8343196404698361
Recall: 0.8393590747665984
Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.89      0.89      2044
           1       0.77      0.79      0.78       977

    accuracy                           0.86      3021
   macro avg       0.83      0.84      0.84      3021
weighted avg       0.86      0.86      0.86      3021



In [215]:
from sklearn.metrics import classification_report

dummy_class = DummyClassifier(strategy="most_frequent")
dummy_class.fit(X_train,y_train)
y_predict=dummy_class.predict(X_test)
y_train_predict=dummy_class.predict(X_train)
Dummy_accuracy = accuracy_score(y_test, y_predict)
Dummy_train_accuracy = accuracy_score(y_train, y_train_predict)
print("Score: ",Dummy_accuracy)
print("training_score :", Dummy_train_accuracy)


print(confusion_matrix(y_test, y_predict))
precision = precision_score(y_test, y_predict, average='macro')
recall = recall_score(y_test, y_predict, average='macro')

print('Precision:', precision)
print('Recall:', recall)

classification_report = classification_report(y_test, y_predict)
print('Classification Report:')
print(classification_report)




Score:  0.6765971532605097
training_score : 0.6682393841569406
[[2044    0]
 [ 977    0]]
Precision: 0.33829857663025487
Recall: 0.5
Classification Report:
              precision    recall  f1-score   support

           0       0.68      1.00      0.81      2044
           1       0.00      0.00      0.00       977

    accuracy                           0.68      3021
   macro avg       0.34      0.50      0.40      3021
weighted avg       0.46      0.68      0.55      3021



/Users/shion/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/shion/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/shion/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/shion/opt/anaconda3/lib/p

In [216]:
from sklearn.metrics import classification_report

dummy_class_over = DummyClassifier(strategy="most_frequent")
dummy_class_over.fit(X_train_oversampled,y_train_oversampled)
y_predict_over=dummy_class_over.predict(X_test)
y_train_predict_over=dummy_class_over.predict(X_train_oversampled)
Dummy_accuracy_over = accuracy_score(y_test, y_predict_over)
Dummy_train_accuracy_over = accuracy_score(y_train_oversampled, y_train_predict_over)
print("Score: ",Dummy_accuracy_over)
print("training_score:", Dummy_train_accuracy_over)

print(confusion_matrix(y_test, y_predict_over))
precision_Over = precision_score(y_test, y_predict_over, average='macro')
recall_Over= recall_score(y_test, y_predict_over, average='macro')

print('Precision:', precision_Over)
print('Recall:', recall_Over)

classification_report = classification_report(y_test, y_predict_over)
print('Classification Report:')
print(classification_report)



Score:  0.6765971532605097
training_score: 0.5
[[2044    0]
 [ 977    0]]
Precision: 0.33829857663025487
Recall: 0.5
Classification Report:
              precision    recall  f1-score   support

           0       0.68      1.00      0.81      2044
           1       0.00      0.00      0.00       977

    accuracy                           0.68      3021
   macro avg       0.34      0.50      0.40      3021
weighted avg       0.46      0.68      0.55      3021



/Users/shion/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/shion/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/shion/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/shion/opt/anaconda3/lib/p

In [225]:
from sklearn.metrics import classification_report

Dec_model_OneR=DecisionTreeClassifier(random_state=42,max_depth=1)
Dec_model_OneR.fit(train_data["SEARCHED_FLAG"].values.reshape(-1, 1), train_data["SUSPECT_ARRESTED_FLAG"])
predictions = Dec_model_OneR.predict(test_data["SEARCHED_FLAG"].values.reshape(-1, 1))
accuracy_OneR = accuracy_score(test_data["SUSPECT_ARRESTED_FLAG"], predictions)

predictions_training=Dec_model_OneR.predict(train_data["SEARCHED_FLAG"].values.reshape(-1, 1))
accuracy_OneR_training=accuracy_score(train_data["SUSPECT_ARRESTED_FLAG"], predictions_training)

print("Accuracy:", accuracy_OneR)
print("Training", accuracy_OneR_training)


print(confusion_matrix(y_test, predictions))
precision = precision_score(y_test, predictions, average='macro')
recall= recall_score(y_test, predictions, average='macro')

print('Precision:', precision)
print('Recall:', recall)

classification_report = classification_report(y_test, predictions)
print('Classification Report:')
print(classification_report)


Accuracy: 0.7871565706719629
Training 0.7859448721132356
[[1571  473]
 [ 170  807]]
Precision: 0.7664118592044802
Recall: 0.797294475480073
Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.77      0.83      2044
           1       0.63      0.83      0.72       977

    accuracy                           0.79      3021
   macro avg       0.77      0.80      0.77      3021
weighted avg       0.81      0.79      0.79      3021



In [218]:
from sklearn.metrics import classification_report

Dec_model_OneR=DecisionTreeClassifier(random_state=42,max_depth=1)
Dec_model_OneR.fit(X_train_oversampled["SEARCHED_FLAG"].values.reshape(-1, 1), y_train_oversampled)
predictions_Over = Dec_model_OneR.predict(test_data["SEARCHED_FLAG"].values.reshape(-1, 1))
accuracy_OneR = accuracy_score(test_data["SUSPECT_ARRESTED_FLAG"], predictions_Over)
print("Accuracy:", accuracy_OneR)

print(confusion_matrix(y_test, predictions_Over))
precision_Over = precision_score(y_test, predictions_Over, average='macro')
recall_Over= recall_score(y_test, predictions_Over, average='macro')

print('Precision:', precision_Over)
print('Recall:', recall_Over)

classification_report = classification_report(y_test, predictions_Over)
print('Classification Report:')
print(classification_report)


Accuracy: 0.7871565706719629
[[1571  473]
 [ 170  807]]
Precision: 0.7664118592044802
Recall: 0.797294475480073
Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.77      0.83      2044
           1       0.63      0.83      0.72       977

    accuracy                           0.79      3021
   macro avg       0.77      0.80      0.77      3021
weighted avg       0.81      0.79      0.79      3021



In [219]:
from sklearn.metrics import classification_report

estimators=[("logreg", logreg), ("rf", model_R), ("Dc", Dec_model),("Naive",Model_N)]
#create our voting classifier, inputting our models
ensemble = VotingClassifier(estimators)
ensemble.fit(X_train, y_train)
#test our model on the test data

y_pred = ensemble.predict(X_test)

print("Voting Classifier",ensemble.score(X_test, y_test))

print(confusion_matrix(y_test, y_pred))
precision = precision_score(y_test, y_pred, average='macro')
recall= recall_score(y_test, y_pred, average='macro')

print('Precision:', precision)
print('Recall:', recall)

classification_report = classification_report(y_test, y_pred)
print("Classification Report:")
print(classification_report)





Voting Classifier 0.8682555445216815
[[1887  157]
 [ 241  736]]
Precision: 0.855468125099984
Recall: 0.8382581667991995
Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.92      0.90      2044
           1       0.82      0.75      0.79       977

    accuracy                           0.87      3021
   macro avg       0.86      0.84      0.85      3021
weighted avg       0.87      0.87      0.87      3021



In [220]:
from sklearn.metrics import classification_report

estimators=[("logreg", logreg), ("rf", model_R), ("Dc", Dec_model),("Naive",Model_N)]
#create our voting classifier, inputting our models
ensemble = VotingClassifier(estimators)
ensemble.fit(X_train_oversampled, y_train_oversampled)
#test our model on the test data
print("Voting Classifier",ensemble.score(X_test, y_test))

y_pred_Over = ensemble.predict(X_test)

print(confusion_matrix(y_test, y_pred_Over))
precision_Over = precision_score(y_test, y_pred_Over, average='macro')
recall_Over= recall_score(y_test, y_pred_Over, average='macro')

print('Precision:', precision_Over)
print('Recall:', recall_Over)


print("Voting Classifier",ensemble.score(X_test, y_test))

classification_report = classification_report(y_test, y_pred_Over)
print("Classification Report:")
print(classification_report)


Voting Classifier 0.8639523336643495
[[1841  203]
 [ 208  769]]
Precision: 0.8448196651181847
Recall: 0.8438941545968228
Voting Classifier 0.8639523336643495
Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.90      0.90      2044
           1       0.79      0.79      0.79       977

    accuracy                           0.86      3021
   macro avg       0.84      0.84      0.84      3021
weighted avg       0.86      0.86      0.86      3021



In [221]:
from sklearn.metrics import classification_report

G_Model=GradientBoostingClassifier(random_state=42) #Gradient Boosted Decision Trees
G_Model.fit(X_train,y_train)
G_predict=G_Model.predict(X_test)
G_predict_train=G_Model.predict(X_train)
G_accracy=accuracy_score(y_test,G_predict)
G_accracy_train=accuracy_score(y_train,G_predict_train)
print("Gradient ", G_accracy)
print("Train Gradient ", G_accracy_train)

print(confusion_matrix(y_test, G_predict))
precision = precision_score(y_test, G_predict, average='macro')
recall= recall_score(y_test, G_predict, average='macro')

print('Precision:', precision)
print('Recall:', recall)


classification_report = classification_report(y_test, G_predict)
print("Classification Report:")
print(classification_report)


Gradient  0.8702416418404502
Train Gradient  0.8659879149077063
[[1882  162]
 [ 230  747]]
Precision: 0.8564403315331532
Recall: 0.8426645528165417
Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.92      0.91      2044
           1       0.82      0.76      0.79       977

    accuracy                           0.87      3021
   macro avg       0.86      0.84      0.85      3021
weighted avg       0.87      0.87      0.87      3021



In [222]:
from sklearn.metrics import classification_report

G_Model=GradientBoostingClassifier(random_state=42) #Gradient Boosted Decision Trees
G_Model.fit(X_train_oversampled,y_train_oversampled)
G_predict=G_Model.predict(X_test)
G_predict_train=G_Model.predict(X_train_oversampled)
G_accracy=accuracy_score(y_test,G_predict)
G_accracy_train=accuracy_score(y_train_oversampled,G_predict_train)
print("Gradient ", G_accracy)
print("Train Gradient ", G_accracy_train)



print(confusion_matrix(y_test, G_predict))
precision = precision_score(y_test, G_predict, average='macro')
recall= recall_score(y_test, G_predict, average='macro')

print('Precision:', precision)
print('Recall:', recall)


classification_report = classification_report(y_test, G_predict)
print("Classification Report:")
print(classification_report)


Gradient  0.8642833498841443
Train Gradient  0.8376687724513812
[[1846  198]
 [ 212  765]]
Precision: 0.8456899448698036
Recall: 0.8430701636664817
Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.90      0.90      2044
           1       0.79      0.78      0.79       977

    accuracy                           0.86      3021
   macro avg       0.85      0.84      0.84      3021
weighted avg       0.86      0.86      0.86      3021



In [223]:
# models = [RandomForestClassifier(random_state=42),RandomForestClassifier(random_state=42),
#             LogisticRegression(random_state=0,max_iter=1000),DecisionTreeClassifier(random_state=42),MultinomialNB()
#          ]
# for model in models:
#     print(model, '\n')
#     score = cross_val_score(model, X_train, y_train, scoring='accuracy', cv=5).mean()
#     print('Cross-Validation Accuracy:', score, '\n', '\n')
